## Introduction

This notebook demonstrates how to use Geometric Median for time dimension reduction. Geomedian computation is quite expensive in terms of memory, data bandwidth and cpu usage. We use Dask to perform data loading and computation in parallel across many threads to speed things up. In this notebook local Dask cluster is used, but the same approach should work using a larger, distributed Dask cluster.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr

## Install missing requirements

```
pip install --user --extra-index-url="https://packages.dea.ga.gov.au" hdstats
pip install --user --extra-index-url="https://packages.dea.ga.gov.au" odc-algo
```

Verify install worked by importing the libraries

In [ ]:
import hdstats
import odc.algo

## Setup local dask cluster

In [ ]:
from datacube.utils.rio import configure_s3_access
from datacube.utils.dask import start_local_dask
import os
import dask
from dask.utils import parse_bytes

# configure dashboard link to go over proxy
dask.config.set({"distributed.dashboard.link":
                 os.environ.get('JUPYTERHUB_SERVICE_PREFIX', '/')+"proxy/{port}/status"});

# Figure out how much memory/cpu we really have (those are set by jupyterhub)
mem_limit = int(os.environ.get('MEM_LIMIT', '0'))
cpu_limit = float(os.environ.get('CPU_LIMIT', '0'))
cpu_limit = int(cpu_limit) if cpu_limit > 0 else 4
mem_limit = mem_limit if mem_limit > 0 else parse_bytes('8Gb')

# leave 4Gb for notebook itself
mem_limit -= parse_bytes('4Gb')

# close previous client if any, so that one can re-run this cell without issues
client = locals().get('client', None)
if client is not None:
    client.close()
    del client
    
client = start_local_dask(n_workers=1,
                          threads_per_worker=cpu_limit, 
                          memory_limit=mem_limit)
display(client)

# Configure GDAL for s3 access 
configure_s3_access(aws_unsigned=True,  # works only when reading public resources
                    client=client);

## Setup Datacube and data source

In this notebook we are using `ls8_ard` and will be computing Geomedian for one Landsat scene (96, 74) using all available observations for the year 2016. To limit computation and memory this example uses only three optical bands (red, green, blue) and we limit computation to a 2K by 2K block of pixels roughly in the middle of the scene.

Cell bellow finds all  the datasets of interest. These all should be in the same projection.

In [ ]:
from datacube import Datacube
from odc.algo import fmask_to_bool, to_f32, from_float, xr_geomedian

dc = Datacube()

product = 'ls8_ard'
region_code, year = '96074', 2016

dss = dc.find_datasets(product=product, 
                       region_code=region_code, 
                       time=str(year))
len(dss)

## Do native load (lazy version with Dask)

In [ ]:
data_bands = ['red', 'green', 'blue']
mask_bands = ['fmask']

xx = dc.load(product=dss[0].type.name,
             output_crs=dss[0].crs,
             resolution=(-30, 30),
             align=(15, 15),
             measurements=data_bands + mask_bands,
             group_by='solar_day',
             datasets=dss, 
             dask_chunks=dict(
                 x=1000, 
                 y=1000)
            )

In [ ]:
# Select a 2k by 2k subsection, to speed up testing
xx = xx.isel(x=np.s_[4000:6000], y=np.s_[4000:6000])

## Compute Geomedian on data_bands
1. Convert fmask to boolean: `True` - use, `False` - do not use
2. Apply masking in native dtype for data bands only
3. Convert to `float32` with scaling
4. Reduce time dimension with geometric median
5. Convert back to native dtype with scaling

All steps are dask operations, so no actual computation is done until `.compute()` is called.

In [ ]:
scale, offset = (1/10_000, 0)  # differs per product, aim for 0-1 values in float32

no_cloud = fmask_to_bool(xx.fmask, ('valid', 'snow', 'water'))

xx_data = xx[data_bands]
xx_clean = odc.algo.keep_good_only(xx_data, where=no_cloud)
xx_clean = to_f32(xx_clean, scale=scale, offset=offset)
yy = xr_geomedian(xx_clean, 
                  num_threads=1,  # disable internal threading, dask will run several concurrently
                  eps=0.2*scale,  # 1/5 pixel value resolution
                  nocheck=True)   # disable some checks inside geomedian library that use too much ram

yy = from_float(yy, 
                dtype='int16', 
                nodata=-999, 
                scale=1/scale, 
                offset=-offset/scale)

## Now we can run the computation

In [ ]:
%%time
yy = yy.compute()

## Convert to RGBA and display

In [ ]:
from odc.ui import to_rgba, to_png_data
from IPython.display import Image

rgba = to_rgba(yy, clamp=3000)
Image(data=to_png_data(rgba.data))

------------------------------------------------------------------